Import libraries

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from numpy import array
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")

Load data

In [2]:
train_df = pd.read_csv("TrainingWiDS2021.csv")
test_df = pd.read_csv("UnlabeledWiDS2021.csv")

Drop unnamed column

In [3]:
train_df = train_df.drop(columns=['Unnamed: 0'])
test_df = test_df.drop(columns=['Unnamed: 0'])

Split training data

In [4]:
X = train_df.drop('diabetes_mellitus', axis=1)
y = train_df['diabetes_mellitus']

kf = KFold(n_splits=5, random_state=42, shuffle=True)

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

One-hot encode categorical variables

In [5]:
X_train

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,...,51.0,51.0,51.0,0,0,0,0,0,0,0
3,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,...,265.0,337.0,337.0,0,0,0,0,0,0,0
4,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,...,NaN,NaN,NaN,0,0,0,0,0,0,0
6,195124,83,59.0,57.451002,0,Caucasian,F,165.1,Operating Room,Accident & Emergency,...,370.0,370.0,370.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130151,147574,7,50.0,20.324301,1,African American,F,165.1,Operating Room,Operating Room / Recovery,...,226.0,226.0,226.0,0,0,0,0,0,0,0
130152,164758,7,50.0,29.287256,0,Caucasian,M,175.3,Emergency Department,Accident & Emergency,...,NaN,NaN,NaN,0,0,0,0,0,0,0
130153,197653,7,79.0,29.653433,0,Caucasian,F,162.6,Direct Admit,Accident & Emergency,...,NaN,NaN,NaN,0,0,0,0,0,0,0
130154,219719,7,73.0,32.265371,0,African American,M,177.8,Emergency Department,Accident & Emergency,...,163.0,163.0,163.0,0,0,0,0,0,0,0


In [6]:
X_train['ethnicity'].fillna('None', inplace=True)
X_train['hospital_admit_source'].fillna('None', inplace=True)
X_train['icu_admit_source'].fillna('None', inplace=True)

In [7]:
X_train['ethnicity']

0                Caucasian
1                Caucasian
3                Caucasian
4                Caucasian
6                Caucasian
                ...       
130151    African American
130152           Caucasian
130153           Caucasian
130154    African American
130155           Caucasian
Name: ethnicity, Length: 104126, dtype: object

In [18]:
len(names)

29

In [22]:
enc = OneHotEncoder(handle_unknown='ignore')

encoded = enc.fit_transform(array(X_train[['ethnicity', 'hospital_admit_source', 'icu_admit_source']])).toarray()
names = enc.get_feature_names(['ethnicity', 'hospital_admit_source', 'icu_admit_source'])
X_train[names] = encoded

Run GBM

In [51]:
# TODO: Drop 3 columns
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)

gradient_booster.fit(X_train, y_train)
print(classification_report(y_val, gradient_booster.predict(X_val)))

ValueError: setting an array element with a sequence.

In [52]:
X_train

,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,...,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
0,214826,118,68.0,22.732803,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",M,180.3,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,NaN,NaN,NaN,0,0,0,0,0,0,0
1,246060,81,77.0,27.421875,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",F,160.0,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,51.0,51.0,51.0,0,0,0,0,0,0,0
3,262220,118,81.0,22.635548,1,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",F,165.1,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,265.0,337.0,337.0,0,0,0,0,0,0,0
4,201746,33,19.0,NaN,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",M,188.0,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,NaN,NaN,NaN,0,0,0,0,0,0,0
6,195124,83,59.0,57.451002,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",F,165.1,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,370.0,370.0,370.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130151,147574,7,50.0,20.324301,1,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",F,165.1,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,226.0,226.0,226.0,0,0,0,0,0,0,0
130152,164758,7,50.0,29.287256,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",M,175.3,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,NaN,NaN,NaN,0,0,0,0,0,0,0
130153,197653,7,79.0,29.653433,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",F,162.6,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,NaN,NaN,NaN,0,0,0,0,0,0,0
130154,219719,7,73.0,32.265371,0,"(0, 2)\t1.0\n (1, 2)\t1.0\n (2, 2)\t1.0\n ...",M,177.8,"(0, 4)\t1.0\n (1, 4)\t1.0\n (2, 9)\t1.0\n ...","(0, 1)\t1.0\n (1, 1)\t1.0\n (2, 3)\t1.0\n ...",...,163.0,163.0,163.0,0,0,0,0,0,0,0
